In [1]:
from process_videos import process_text_video_with_flightaware, extract_text_from_video_av
from pathlib import Path
from typing import List
import pandas as pd
import json

/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.
No model hoster is available! Please check your network connection to one of the following model hoster: HuggingFace (https://huggingface.co), ModelScope (https://modelscope.cn), AIStudio (https://aistudio.baidu.com), or BOS (https://paddle-model-ecology.bj.bcebos.com). Otherwise, only local models can be used.
/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. 

In [2]:

def find_video_files(directory: str, recursive: bool = True) -> List[str]:
    """
    Busca archivos de video en un directorio de forma recursiva.
    
    Args:
        directory: Ruta del directorio donde buscar los videos
        recursive: Si es True, busca en subdirectorios de forma recursiva
        
    Returns:
        Lista con las rutas completas de los archivos de video encontrados
    """
    # Extensiones de video más comunes
    video_extensions = [
        '.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.mpeg', '.mpg',
        '.m4v', '.webm', '.3gp', '.3g2', '.m2ts', '.mts', '.ts', '.m2v',
        '.mxf', '.ogv', '.vob', '.asf', '.rm', '.rmvb', '.f4v', '.m4p',
        '.m4v', '.svi', '.m4b', '.m4r', '.3gpp', '.3g2', '.mxf'
    ]
    
    video_files = []
    path = Path(directory)
    
    if recursive:
        # Búsqueda recursiva
        for ext in video_extensions:
            video_files.extend([str(p) for p in path.rglob(f'*{ext}')])
            # Incluir también versiones en mayúsculas
            video_files.extend([str(p) for p in path.rglob(f'*{ext.upper()}')])
    else:
        # Búsqueda solo en el directorio actual
        for ext in video_extensions:
            video_files.extend([str(p) for p in path.glob(f'*{ext}')])
            video_files.extend([str(p) for p in path.glob(f'*{ext.upper()}')])
    
    return sorted(list(set(video_files)))  # Eliminar duplicados y ordenar


In [ ]:
videos_path = find_video_files("/home/gabo/Personal/Work/Lita/Proyectos/VideoSearch/vid-chronicle-main/data/videos/TWR-HAV")
videos_results = []
df = pd.DataFrame()

for vp in videos_path:
    if "+0,0" not in vp:
        continue

    result = extract_text_from_video_av(vp, 360)

    if result:
        videos_results.append(result)
        df = pd.concat([df, process_text_video_with_flightaware(result)], ignore_index=True)


with open("ocr_results_new.json", 'w', encoding='utf-8') as f:
    json.dump(videos_results, f, ensure_ascii=False, indent=2)

df = df.sort_values(by=["date", "time", "codigo_vuelo"]) 
df.to_csv("data/flightaware_results_new.csv", index=False)


In [ ]:
vp = Path(videos_path[0])
vp.name.replace(".mkv", ".json")


'2025-05-12T16:29:08Z_2025-05-12T16:44:08Z.mkv'

In [17]:
videos_path = find_video_files("/home/gabo/Personal/Work/Lita/Proyectos/VideoSearch/vid-chronicle-main/data/videos/TWR-HAV")
videos_results = []

for vp in videos_path:
    vp_path = Path(vp)

    if "+0,0" not in vp:
        continue

    result = extract_text_from_video_av(vp, 360)
    videos_results.append(result)

    with open(f"data/ocr/{vp_path.name.replace('.mkv', '.json')}", 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    df_actual = pd.DataFrame(process_text_video_with_flightaware(result))
    df_actual.to_csv(f"data/db/{vp_path.name.replace('.mkv', '.csv')}", index=False)
    df = pd.concat([df, df_actual], ignore_index=True)

with open("ocr_results.json", 'w', encoding='utf-8') as f:
    json.dump(videos_results, f, ensure_ascii=False, indent=2)

df = df.sort_values(by=["date", "time", "codigo_vuelo"]) 
df.to_csv("data/flightaware_results.csv", index=False)



🎬 Procesando video: /home/gabo/Personal/Work/Lita/Proyectos/VideoSearch/vid-chronicle-main/data/videos/TWR-HAV/2025/05/12/+0,0/2025-05-12T16:29:08Z_2025-05-12T16:44:08Z.mkv
Tiempo: 0.00s | Texto: System View Lists Alerts Msgs Help 16:29:08 NUEVAS-EXE a d C 5460 330 AAL1519 n MUCU 250250 2 OFFR . OFFR o JBU1793 2 . CSQ503 2 M 330330 149150 A320/M N462* 0O SF34/M N268 MROC o MKJS AVA009 1 M D AVA255 2 M 3571370 370370 SKBO 0 SKBO A20N/M N468 OFFR CMP484 2 C B39M/M N443 360360 ABX22451M 350350 MPTO SKBO o D:23.5 196 Tr01:1 OFFR CMP357 2 M NKS723 2 M 380380 370370 GHANN 1632 F388 8738/M N443 A321/M N475 MPTO MKJP AVA036 2 A 360360  Fixed A320/M N439 KFLL VESDA 163R F3R0 CMP471 2 M SWG6147 47 A 350350 023↑160 B39M/M N467 DAL1997 2 M CYUL MPTO 330330 o 0 A321/M N464 MKJS AAL2217 CMP312 . o DAL980 2 C 350 370 360360 B752/M N444 0 KATL ] FDX205 2 M 371370 8763/H N483 SKBO UCU 1649 F380 SASUS 1653F388 C n C ACA099 2 C 380380 CYUuL A333/H N448
Tiempo: 353.20s | Texto: System View Lists Alerts Ms